# TFG: Network properties of Informal Support Networks in Mental Health

Oriol Martinez Perez

We will work with data set https://zenodo.org/record/7443141 that was created within the CoAct project (as described there). The data is stored in folder data/ within the same folder as this jupyter notebook. 

## 1. Load some data

We will work with python, and more specifically with python pandas.

In [3]:
import pandas as pd

First, we load the answers of the participants to stories type C (See documentation on Zenodo, https://zenodo.org/record/7443141)

In [4]:
df_C = pd.read_pickle("./data/Answers_to_Microstories/answers_stories_C.pkl")
df_C

0       capacitation_Teatre_Amigues                                          \
1                              yo_1  yo_2 yo_final otro_1 otro_2 otro_final   
f91cb3f                           B  None        B      C   None          C   
65fc038                           C  None        C      C   None          C   
5a20153                           C  None        C      C   None          C   
9f4d288                           C  None        C      C   None          C   
a1efa63                           A  None        A      B   None          B   
...                             ...   ...      ...    ...    ...        ...   
024f546                           C  None        C      C   None          C   
190aebe                        None  None     None   None   None       None   
4a92bfd                           C  None        C      C   None          C   
388a24f                           C  None        C      C   None          C   
c2d349a                        None  None     None   None   None       None   

0       Obrir_camí                        ... Incomprensió_1             \
1             yo_1  yo_2 yo_final otro_1  ...         otro_2 otro_final   
f91cb3f          A  None        A      C  ...           None       None   
65fc038       None  None     None   None  ...           None       None   
5a20153          A  None        A      A  ...           None       None   
9f4d288          A  None        A      A  ...           None       None   
a1efa63          A  None        A      A  ...           None       None   
...            ...   ...      ...    ...  ...            ...        ...   
024f546       None  None     None   None  ...           None       None   
190aebe       None  None     None   None  ...           None       None   
4a92bfd          C  None        C      C  ...           None       None   
388a24f       None  None     None   None  ...           None       None   
c2d349a       None  None     None   None  ...           None       None   

0       Por_situacions_socials                             \
1                         yo_1 yo_final otro_1 otro_final   
f91cb3f                   None     None   None       None   
65fc038                   None     None   None       None   
5a20153                   None     None   None       None   
9f4d288                   None     None   None       None   
a1efa63                   None     None   None       None   
...                        ...      ...    ...        ...   
024f546                   None     None   None       None   
190aebe                   None     None   None       None   
4a92bfd                   None     None   None       None   
388a24f                   None     None   None       None   
c2d349a                   None     None   None       None   

0       Reconeixement_comprensio                             
1                           yo_1 yo_final otro_1 otro_final  
f91cb3f                     None     None   None       None  
65fc038                     None     None   None       None  
5a20153                     None     None   None       None  
9f4d288                     None     None   None       None  
a1efa63                     None     None   None       None  
...                          ...      ...    ...        ...  
024f546                     None     None   None       None  
190aebe                     None     None   None       None  
4a92bfd                     None     None   None       None  
388a24f                     None     None   None       None  
c2d349a                     None     None   None       None  

[748 rows x 706 columns]

The DataFrame we loaded is a pandas MultiIndex DataFrame. We are not interested in there intermediate decisions, but only in the FINAL decisions of the participants, so we reduce the DataFrame to columns yo_final and otro_final. Particularly, we create two DataFrames with only these columns.

In [7]:
df_C.columns= df_C.columns.swaplevel(0,1)
df_C_yo   = df_C.yo_final
df_C_otro = df_C.otro_final
df_C.columns= df_C.columns.swaplevel(0,1)

df_C_yo

,capacitation_Teatre_Amigues,Obrir_camí,Experiencia_aprenentatge,Gossos,Sanglotant,Empatia_comprensio,Encuentrobis,Ataque_ansiedad_b,Temps_aprendre,Bipolar,...,Joys_motherhood_2,Esperança,Al·lucinacions,Escoltar_temps,Germans,Suport,Realitat_ficció,Incomprensió_1,Por_situacions_socials,Reconeixement_comprensio
f91cb3f,B,A,A,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
65fc038,C,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5a20153,C,A,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9f4d288,C,A,B,C,B,A,B,C,B,None,...,None,None,None,None,None,None,None,None,None,None
a1efa63,A,A,B,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
024f546,C,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
190aebe,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4a92bfd,C,C,A,C,C,A,C,None,None,None,...,None,None,None,None,None,None,None,None,None,None
388a24f,C,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [8]:
df_C_yo.describe()

,capacitation_Teatre_Amigues,Obrir_camí,Experiencia_aprenentatge,Gossos,Sanglotant,Empatia_comprensio,Encuentrobis,Ataque_ansiedad_b,Temps_aprendre,Bipolar,...,Joys_motherhood_2,Esperança,Al·lucinacions,Escoltar_temps,Germans,Suport,Realitat_ficció,Incomprensió_1,Por_situacions_socials,Reconeixement_comprensio
count,580,399,361,332,298,281,248,238,225,204,...,21,21,21,21,20,20,20,20,20,19
unique,3,3,3,3,3,3,3,3,3,3,...,3,3,2,3,3,3,3,3,3,3
top,C,A,A,C,C,B,B,B,A,B,...,C,C,C,C,C,C,C,C,C,C
freq,391,280,152,183,203,120,87,96,104,86,...,13,14,16,9,14,11,12,12,9,11


Secondly, we also load the answers of the (same) participants to the sociodemographic survey (columns documented in ./data/Sociodemographic_survey/meta_data_answers.pdf):

In [10]:
df_sociodem = pd.read_pickle("./data/Sociodemographic_survey/answers_sociodemographic_survey.pkl")
df_sociodem

1,genere,edat,pp,pp_v,pp_v_g,pp_bf,pp_bef,pp_bel,pp_bev,pp_bec,...,p_ac_si,p_g1,p_g2,p_ass,p_prof,p_prof_si,country_alpha2,urban_rural,is_in_Catalonia,language
f91cb3f,f,4,Sí,Sí,a,m,a,a,b,b,...,None,b,m,Sí,No,None,US,None,False,en
65fc038,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,False,ca
5a20153,f,3,No,None,None,None,None,None,None,None,...,None,a,a,Sí,No,None,ES,urban,True,ca
9f4d288,m,4,No,None,None,None,None,None,None,None,...,None,b,b,Sí,Sí,apasa,ES,urban,True,ca
a1efa63,f,2,Sí,Sí,nsdmp,b,m,nsdmp,nvr,nvr,...,nvr,a,m,No,nvr,None,ES,None,False,es
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
024f546,f,1,Sí,Sí,a,m,a,nt,nsdmp,m,...,None,b,m,Sí,Sí,psm,ES,urban,True,ca
190aebe,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,False,ca
4a92bfd,m,4,Sí,Sí,m,m,b,nsdmp,nsdmp,nsdmp,...,None,a,None,No,No,None,ES,None,False,es
388a24f,f,4,Sí,Sí,a,a,a,m,b,b,...,None,a,None,None,None,None,ES,urban,True,ca


**TASK**: Find out how many women and how many man are in the sample.

In [11]:
df_sociodem['genere'].value_counts()

f      366
m      155
d       11
nvr      3
Name: genere, dtype: int64

In [13]:
df_sociodem['country_alpha2'].value_counts()

ES    463
US     17
DE     12
MX      3
AT      3
AR      3
FI      2
BE      2
PT      2
IL      1
CH      1
FR      1
IT      1
CA      1
Name: country_alpha2, dtype: int64